**setup**

In [43]:
#get gpu hosts
hosts = [line.strip() for line in open('psanagpu.txt')]
#hosts=hosts[:2]+hosts[3:7]+hosts[8:]
hosts=['psanagpu102', 'psanagpu105', 'psanagpu106', 'psanagpu110', 'psanagpu112', 'psanagpu114', 'psanagpu115']

In [44]:
basepath="/reg/neh/home5/zimmf/idi"

**create commands**

In [45]:
from collections import deque
commands=deque()

In [46]:
# Usage: sim.py [options] sphere/gridsc/gridfcc/gridblocks

# Options:
#   -h, --help            show this help message and exit
#   -d NDET, --Ndet=NDET  Number of pixels on detector (default: 512)
#   -r R                  radius in nm (default: 50)
#   -a A                  lattice constant in A (default: 5)
#   -n NATOMS, --Natoms=NATOMS
#                         Number of atoms (default: 1e5)
#   -p PIXELSIZE, --pixelsize=PIXELSIZE
#                         Size of pixels on detector in um (default: 75)
#   -z DETZ, --detz=DETZ  z distance of detector in cm(default: 10)
#   -i NIMG, --Nimg=NIMG  Number of images (default: 100)
#   -e E, -E E            Energy in eV (default: 1500)
#   -c, --coherent        no random phases
#   -o OUTFILE, --output=OUTFILE
#                         output file (default: out.npz)
#   --Nunitcells=NUNITCELLS
#                         grid: nx,ny,nz Number of unitcells in x y and z
#                         direction, overrides Natoms (default: use Natoms)
#   --anglex=AX           grid: Rotation angle in degree (default: 0)
#   --angley=AY           grid: Rotation angle in degree (default: 0)
#   --anglez=AZ           grid: Rotation angle in degree (default: 0)
#   -f, --fixedpositions  sphere: no randomly changing positions
#   --nocuda              dont use cuda

In [47]:
#yes, this is the right place to change the settings
#ugly but easy for fast changes ;)

#in this example we'll use 10 batches of 100 images each.
#a lattice constant of 5A, 1e3 Atoms at fixed positions, a detector distance of 4cm, 150um pixelsize, 512x512 pixel,
#the fluorescence energy is 9kev and we're doing a sc grid

#you should change the path as well but keep the format filename-(2digit number).npz!

#for i in range(27): #number of batches
#    arguments='-i 100 -a 3.6 -n 1e3 -f -z 4 -p 150 -d 512 -E 9000  -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/test/test-%02d.npz gridsc'%i
#    commands.append("%s/pythonwrapper.sh %s/sim.py %s" % (basepath,basepath,arguments) )


In [48]:
for i in range(28): #number of batches
    arguments='--anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/sacla/gaas_rotx10-%02d.npz gridfcc'%i
    commands.append("%s/pythonwrapper.sh local3 %s/idi_sim.py %s" % (basepath,basepath,arguments) )

In [49]:
commands

deque(['/reg/neh/home5/zimmf/idi/pythonwrapper.sh local3 /reg/neh/home5/zimmf/idi/idi_sim.py --anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/sacla/gaas_rotx10-00.npz gridfcc',
       '/reg/neh/home5/zimmf/idi/pythonwrapper.sh local3 /reg/neh/home5/zimmf/idi/idi_sim.py --anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/sacla/gaas_rotx10-01.npz gridfcc',
       '/reg/neh/home5/zimmf/idi/pythonwrapper.sh local3 /reg/neh/home5/zimmf/idi/idi_sim.py --anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/sacla/gaas_rotx10-02.npz gridfcc',
       '/reg/neh/home5/zimmf/idi/pythonwrapper.sh local3 /reg/neh/home5/zimmf/idi/idi_sim.py --anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/sacla/gaas_rotx10-03.npz gridfcc',


**run commands**

In [ ]:
#simple queueing system for the gpuhosts

import subprocess
import sys
from time import sleep
sshs=len(hosts)*[None]
while True:
    for n,(ssh,host) in enumerate(zip(sshs,hosts)):
        if ssh is None or ssh.poll()==0:
            if ssh is not None:
                print ('%s:%s'%(host, ssh.stdout.readlines()))
            if len(commands):
                command=commands.popleft()
                print('starting \n"%s"\n on %s'%(command,host))
                ssh=subprocess.Popen(["ssh", "%s" % host, "%s" % command],
                                   shell=False,
                                   stdout=subprocess.PIPE,
                                   stderr=subprocess.PIPE)        
            else:
                ssh = None
        sshs[n]=ssh
    sleep(1)
    
    if all(s is None for s in sshs): break

starting 
"/reg/neh/home5/zimmf/idi/pythonwrapper.sh local3 /reg/neh/home5/zimmf/idi/idi_sim.py --anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/sacla/gaas_rotx10-00.npz gridfcc"
 on psanagpu102
starting 
"/reg/neh/home5/zimmf/idi/pythonwrapper.sh local3 /reg/neh/home5/zimmf/idi/idi_sim.py --anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/sacla/gaas_rotx10-01.npz gridfcc"
 on psanagpu105
starting 
"/reg/neh/home5/zimmf/idi/pythonwrapper.sh local3 /reg/neh/home5/zimmf/idi/idi_sim.py --anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/amolr9316/scratch/zimmf/sacla/gaas_rotx10-02.npz gridfcc"
 on psanagpu106
starting 
"/reg/neh/home5/zimmf/idi/pythonwrapper.sh local3 /reg/neh/home5/zimmf/idi/idi_sim.py --anglex 5 -i 50 -a 5.6 --Nunitcells 100,100,100 -f -z 4 -p 25 -d 1024 -E 9250 -o /reg/d/psdm/amo/am

In [ ]:
#getting error messages for debuging
for ssh in sshs:
    if ssh is not None:
        result = ssh.stdout.readlines()
        if result == []:
            error = ssh.stderr.readlines()
            print("ERROR: %s" % error)
        else:
            print result
    else:
        print('done')




**handy one liner to check on the simulation** :

`while read i; do; echo $i;ssh -n $i nvidia-smi|grep python; done <psanagpu.txt`